In [ ]:
import nltk
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from sklearn import feature_extraction

In [ ]:

from google.colab import drive
drive.mount("/content/gdrive")


Mounted at /content/gdrive


In [ ]:
dataset = pd.read_csv('/content/gdrive/My Drive/NLPlay with Transformers/Sentiment analysis/IMDB Dataset.csv')
dataset.info()
dataset.head(n=5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
one_hot = pd.get_dummies(dataset['sentiment'])
dataset = dataset.drop('sentiment', axis=1) 
dataset = dataset.join(one_hot)
dataset.head(5)

,review,negative,positive
0,One of the other reviewers has mentioned that ...,0,1
1,A wonderful little production. <br /><br />The...,0,1
2,I thought this was a wonderful way to spend ti...,0,1
3,Basically there's a family where a little boy ...,1,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",0,1


In [ ]:
from sklearn.model_selection import train_test_split
X =dataset.iloc[:, 0].values
Y = dataset.iloc[:, 2].values


Tranformers


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.5MB 4.0MB/s 
     |████████████████████████████████| 901kB 33.0MB/s 
     |████████████████████████████████| 3.3MB 42.3MB/s 


In [ ]:
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
dataset=[]
for i in range(len(X)):
  encoding = tokenizer.encode_plus(
  X[i],
  truncation=True,
  max_length=64,
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=False,
  padding='max_length',
  return_attention_mask=True,
  return_tensors='pt',  # Return PyTorch tensors
)
  dataset.append({
        'input_ids': encoding['input_ids'].flatten(),
        'attention_mask': encoding['attention_mask'].flatten(),
        'targets': torch.tensor(Y[i], dtype=torch.long)})


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test= train_test_split(dataset, shuffle=True, random_state=60, test_size=0.20)
print("Number of train samples: ", len(X_train))
print("Number of test samples: ", len(X_test))
from torch.utils.data import DataLoader
train_dataloader = DataLoader(X_train, batch_size=32, shuffle=True)
test_dataloader = DataLoader(X_test, batch_size=32, shuffle=True)

Number of train samples:  40000
Number of test samples:  10000


In [ ]:
class robertaClassifier(nn.Module):

  def __init__(self):
    super(robertaClassifier, self).__init__()
    self.roberta = AutoModel.from_pretrained("roberta-base")
    self.drop = nn.Dropout(0.3)
    self.x= nn.Linear(self.roberta.config.hidden_size, 1)
    self.out=nn.Sigmoid()
  
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.roberta(
      input_ids=input_ids,
      attention_mask=attention_mask,return_dict=False)
    out1= self.drop(pooled_output)
    out2=self.x(out1)
    return self.out(out2)

In [ ]:
model = robertaClassifier()
n_epocs=3
n_samples=40000
batch_size=32
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=3e-5)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
for n in range(1,n_epocs):
  correct=0
  train_loss=0
  batch=0
  for d in train_dataloader:
    batch+=1
    optimizer.zero_grad()
    input_ids = d["input_ids"]
    attention_mask = d["attention_mask"]
    target = d["targets"]
    target=target.to(torch.float32)
    output = model(input_ids=input_ids, attention_mask=attention_mask)
    loss = criterion(output[:,0],target)
    with torch.no_grad():
      train_loss+=loss
      output = (output>0.5).float()
      correct += ((output[:,0] == target).float()).sum()
      print("batches_trained: " + str(batch)+" total_loss: "+str(train_loss/(batch*batch_size))+" acc: "+str(correct/(batch*batch_size)))
    loss.backward()
    optimizer.step()
  print("Epoch completed: " + str(n)+" total_loss: "+str(train_loss)+" acc: "+str(correct/n_samples))
  

In [ ]:
torch.save(model.state_dict(), '/content/gdrive/My Drive/NLPlay with Transformers/Sentiment analysis/model.pt')

In [ ]:
batch=0
for d in test_dataloader:
  batch+=1
  with torch.no_grad():
    input_ids = d["input_ids"]
    attention_mask = d["attention_mask"]
    target = d["targets"]
    target=target.to(torch.float32)
    output = model(input_ids=input_ids, attention_mask=attention_mask)
    output = (output>0.5).float()
    correct += ((output[:,0] == target).float()).sum()
print("acc: "+str(correct/(batch*batch_size)))
    